In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [2]:
test_group = pd.read_csv('report-1525674019.csv')
control_group = pd.read_csv('report-1525674036.csv')
old_reg_group = pd.read_csv('report-1525674044.csv')
test_group['group'] = 'test_group'
control_group['group'] = 'control_group'
old_reg_group['group'] = 'old_reg_group'
df= pd.concat([test_group, control_group, old_reg_group])

In [3]:
df.head()

,user_id,sex,isReferral,isPaid,active_days_0_1day,active_days_1day_2day,active_days_2day_3day,like_sent_0_1day,like_sent_1day_2day,like_sent_2day_3day,fake_like_sent_0_1day,fake_like_sent_1day_2day,fake_like_sent_2day_3day,like_received_0_1day,like_received_1day_2day,like_received_2day_3day,fake_like_received_0_1day,fake_like_received_1day_2day,fake_like_received_2day_3day,incoming_bomb_0_1day,incoming_bomb_1day_2day,incoming_bomb_2day_3day,message_sent_0_1day,message_sent_1day_2day,message_sent_2day_3day,message_received_0_1day,message_received_1day_2day,message_received_2day_3day,mutual_0_1day,mutual_1day_2day,mutual_2day_3day,fake_mutual_0_1day,fake_mutual_1day_2day,fake_mutual_2day_3day,incoming_fake_dialog_0_1day,incoming_fake_dialog_1day_2day,incoming_fake_dialog_2day_3day,outgoing_fake_dialog_0_1day,outgoing_fake_dialog_1day_2day,outgoing_fake_dialog_2day_3day,fist_dialog_received_0_1day,fist_dialog_received_1day_2day,fist_dialog_received_2day_3day,first_dialog_started_0_1day,first_dialog_started_1day_2day,first_dialog_started_2day_3day,fist_dialog_received_fake_0_1day,fist_dialog_received_fake_1day_2day,fist_dialog_received_fake_2day_3day,first_dialog_started_fake_0_1day,first_dialog_started_fake_1day_2day,first_dialog_started_fake_2day_3day,payment_0_1day,payment_1day_2day,payment_2day_3day,group
0,118402819,1,1,0,1,0,0,31,0,0,0,0,0,0,0,0,20,40,11,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,test_group
1,118217623,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,test_group
2,118305379,0,0,0,1,1,1,263,9,0,178,0,0,239,234,91,0,0,0,2,1,1,76,32,0,116,42,12,54,15,7,16,0,0,0,0,0,0,0,0,25,12,8,5,2,0,7,0,0,0,0,0,0,0,0,test_group
3,118401254,1,0,0,1,0,0,22,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,test_group
4,118213887,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,test_group


In [4]:
'user_id','sex','isReferral','isPaid','active_days','like_sent','fake_like_sent','like_received','fake_like_received',
'incoming_bomb','message_sent','message_received','mutual','fake_mutual','incoming_fake_dialog','outgoing_fake_dialog',
'fist_dialog_received','first_dialog_started','fist_dialog_received_fake','first_dialog_started_fake','payment'

('fist_dialog_received',
 'first_dialog_started',
 'fist_dialog_received_fake',
 'first_dialog_started_fake',
 'payment')

In [5]:
periods = ['_0_1day','_1day_2day','_2day_3day']
sex_dict = {0: 'women', 1: 'men'}
groups = ['test_group','control_group','old_reg_group']
metrics = ['active_days','like_sent','fake_like_sent','like_sent_nonfake','like_received','fake_like_received',
           'like_received_nonfake','incoming_bomb','mutual','fake_mutual','mutual_nonfake',
           'message_sent','outgoing_fake_dialog','message_sent_nonfake',
           'message_received','incoming_fake_dialog','message_received_nonfake',
           'fist_dialog_received','fist_dialog_received_fake','fist_dialog_received_nonfake',
           'first_dialog_started','first_dialog_started_fake','first_dialog_started_nonfake','payment']

metrics = ['active_days','like_sent','fake_like_sent','like_received','fake_like_received','incoming_bomb',
          'mutual','fake_mutual','mutual_nonfake','message_sent','message_sent_nonfake','message_received',
          'fist_dialog_received','first_dialog_started','payment']

for period in periods:
    df['like_sent_nonfake'+period] = df['like_sent'+period] - df['fake_like_sent'+period]
    df['like_received_nonfake'+period] = df['like_received'+period] - df['fake_like_received'+period]
    df['mutual_nonfake'+period] = df['mutual'+period] - df['fake_mutual'+period]
    df['message_received_nonfake'+period] = df['message_received'+period] - df['incoming_fake_dialog'+period]
    df['message_sent_nonfake'+period] = df['message_sent'+period] - df['outgoing_fake_dialog'+period]
    df['fist_dialog_received_nonfake'+period] = df['fist_dialog_received'+period] - df['fist_dialog_received_fake'+period]
    df['first_dialog_started_nonfake'+period] = df['first_dialog_started'+period] - df['first_dialog_started_fake'+period]

df = df[['group','user_id','sex','isReferral','isPaid',*[metrix + period for metrix in metrics for period in periods]]]

In [6]:
out = pd.DataFrame(index =[i+'_'+j for j in ['sign','part','mean','quantile25','quantile50','quantile75'] for i in metrics] )
for group in groups:
    for sex in [0, 1]:
        for period in periods:
            d = df[(df['group'] == group) & (df['sex'] == sex) & (df['active_days' + period] == 1)].copy()
            d = d[[x for x in df.columns if period in x]]
            d = pd.concat( [d.agg(lambda x: np.sum(np.sign(abs(x)))), d.agg(lambda x: np.sum(np.sign(abs(x)))) / d.shape[0], d.mean(), d.quantile(0.25), d.quantile(0.5), d.quantile(0.75)], axis = 0)
            d.index = [i+'_'+j for j in ['sign','part','mean','quantile25','quantile50','quantile75'] for i in metrics]
            out = pd.concat([out, d], axis = 1)
out.columns = ['_'.join([group,sex,period]) for group in groups for sex in ['women','men'] for period in ['0','1','2']]
out = out[['_'.join([group,sex,period]) for sex in ['women','men'] for period in ['0','1','2'] for group in groups]]

In [7]:
out.head()

,test_group_women_0,control_group_women_0,old_reg_group_women_0,test_group_women_1,control_group_women_1,old_reg_group_women_1,test_group_women_2,control_group_women_2,old_reg_group_women_2,test_group_men_0,control_group_men_0,old_reg_group_men_0,test_group_men_1,control_group_men_1,old_reg_group_men_1,test_group_men_2,control_group_men_2,old_reg_group_men_2
active_days_sign,1502.0,1559.0,1523.0,649.0,632.0,660.0,420.0,410.0,421.0,13081.0,13060.0,13096.0,3771.0,3802.0,3889.0,2295.0,2322.0,2407.0
like_sent_sign,736.0,794.0,767.0,304.0,295.0,302.0,130.0,140.0,144.0,8668.0,8669.0,8654.0,1980.0,2016.0,2091.0,1002.0,1056.0,1091.0
fake_like_sent_sign,1119.0,1198.0,1156.0,449.0,452.0,476.0,208.0,201.0,206.0,668.0,671.0,699.0,455.0,468.0,480.0,234.0,260.0,250.0
like_received_sign,1080.0,1160.0,1118.0,554.0,557.0,566.0,361.0,350.0,373.0,1106.0,1123.0,1095.0,769.0,824.0,813.0,446.0,445.0,454.0
fake_like_received_sign,159.0,166.0,166.0,134.0,129.0,153.0,90.0,88.0,91.0,2655.0,2619.0,2597.0,1937.0,1927.0,1943.0,1256.0,1272.0,1299.0


In [8]:
writer = pd.ExcelWriter('Multirun_FirstSessionSearch_PositiveActivity_Mobile_run1_1.xlsx')
out.to_excel(writer,'group_sex_period') 
writer.save()

In [9]:
out

,test_group_women_0,control_group_women_0,old_reg_group_women_0,test_group_women_1,control_group_women_1,old_reg_group_women_1,test_group_women_2,control_group_women_2,old_reg_group_women_2,test_group_men_0,control_group_men_0,old_reg_group_men_0,test_group_men_1,control_group_men_1,old_reg_group_men_1,test_group_men_2,control_group_men_2,old_reg_group_men_2
active_days_sign,1502.000000,1559.000000,1523.000000,649.000000,632.000000,660.000000,420.000000,410.000000,421.000000,13081.000000,13060.000000,13096.000000,3771.000000,3802.000000,3889.000000,2295.000000,2322.000000,2407.000000
like_sent_sign,736.000000,794.000000,767.000000,304.000000,295.000000,302.000000,130.000000,140.000000,144.000000,8668.000000,8669.000000,8654.000000,1980.000000,2016.000000,2091.000000,1002.000000,1056.000000,1091.000000
fake_like_sent_sign,1119.000000,1198.000000,1156.000000,449.000000,452.000000,476.000000,208.000000,201.000000,206.000000,668.000000,671.000000,699.000000,455.000000,468.000000,480.000000,234.000000,260.000000,250.000000
like_received_sign,1080.000000,1160.000000,1118.000000,554.000000,557.000000,566.000000,361.000000,350.000000,373.000000,1106.000000,1123.000000,1095.000000,769.000000,824.000000,813.000000,446.000000,445.000000,454.000000
fake_like_received_sign,159.000000,166.000000,166.000000,134.000000,129.000000,153.000000,90.000000,88.000000,91.000000,2655.000000,2619.000000,2597.000000,1937.000000,1927.000000,1943.000000,1256.000000,1272.000000,1299.000000
incoming_bomb_sign,984.000000,1045.000000,1024.000000,396.000000,384.000000,409.000000,221.000000,211.000000,228.000000,5382.000000,5423.000000,5494.000000,2274.000000,2231.000000,2342.000000,1533.000000,1548.000000,1606.000000
mutual_sign,913.000000,979.000000,972.000000,509.000000,488.000000,507.000000,267.000000,243.000000,283.000000,2011.000000,2076.000000,2099.000000,1150.000000,1204.000000,1258.000000,585.000000,632.000000,679.000000
fake_mutual_sign,809.000000,852.000000,851.000000,234.000000,213.000000,240.000000,82.000000,72.000000,81.000000,308.000000,282.000000,286.000000,326.000000,322.000000,338.000000,165.000000,186.000000,184.000000
mutual_nonfake_sign,556.000000,619.000000,578.000000,470.000000,454.000000,472.000000,250.000000,232.000000,263.000000,1862.000000,1923.000000,1946.000000,1024.000000,1064.000000,1138.000000,509.000000,553.000000,607.000000
message_sent_sign,668.000000,700.000000,671.000000,396.000000,384.000000,396.000000,211.000000,207.000000,211.000000,2777.000000,2880.000000,2889.000000,1271.000000,1293.000000,1382.000000,552.000000,596.000000,614.000000


In [10]:
for metrix in metrics:
    df[metrix] = df[metrix + periods[0]] + df[metrix + periods[1]] + df[metrix + periods[2]]

out = pd.DataFrame(index =[i+'_'+j for j in ['sign','part','mean','quantile25','quantile50','quantile75'] for i in metrics] )

for group in groups:
    for sex in [0, 1]:
        d = df[(df['group'] == group) & (df['sex'] == sex)].copy()
        d = d[[x for x in df.columns if x in metrics]]
        d = pd.concat( [d.agg(lambda x: np.sum(np.sign(abs(x)))), d.agg(lambda x: np.sum(np.sign(abs(x)))) / d.shape[0], d.mean(), d.quantile(0.25), d.quantile(0.5), d.quantile(0.75)], axis = 0)
        d.index = [i+'_'+j for j in ['sign','part','mean','quantile25','quantile50','quantile75'] for i in metrics]
        out = pd.concat([out, d], axis = 1)
out.columns = ['_'.join([group,sex]) for group in groups for sex in ['women','men']]
out = out[['_'.join([group,sex]) for sex in ['women','men'] for group in groups]]

writer = pd.ExcelWriter('Multirun_FirstSessionSearch_PositiveActivity_Mobile_run1_2.xlsx')
out.to_excel(writer,'group_sex') 
writer.save()

In [11]:
df[(df.group == 'test_group') & (df.sex == 0) & (df.mutual_0_1day > 0)].shape[0] / df[(df.group == 'test_group') & (df.sex == 0)].shape[0]

0.6028552887735237

In [12]:
df[(df.group == 'control_group') & (df.sex == 0) & (df.mutual_0_1day > 0)].shape[0] / df[(df.group == 'control_group') & (df.sex == 0)].shape[0]

0.6210131332082551

In [13]:
df[(df.group == 'old_reg_group') & (df.sex == 0) & (df.mutual_0_1day > 0)].shape[0] / df[(df.group == 'old_reg_group') & (df.sex == 0)].shape[0]

0.6326142131979695